In [68]:
# T2-1. Titanic (Classification) / 타이타닉
# 학습용 데이터 (X_train, y_train)을 이용하여 생존 예측 모형을 만든 후, 
# 이를 평가용 데이터(X_test)에 적용하여 얻은 예측값을 다음과 같은 형식의 CSV파일로 생성하시오
# (제출한 모델의 성능은 accuracy 평가지표에 따라 채점)
# 풀이

import pandas as pd

train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

# EDA
# print(train.head(3))
# print(train.isnull().sum()) # Age(177), Cabin(687), Embarked(2)
# print(train.shape) # (891, 12)

# print(test.head(3))
# print(test.isnull().sum()) # Age(86), Cabin(327), Fare(1)
# print(test.shape) # (418, 11) Survived 빠짐

# 결측치 처리
y_train = train[['Survived']]
# print(y_train)

train = train.drop(columns=['Survived'])
# print(train.head())
# print(test.head())

df = pd.concat([train, test])
# print(help(pd.concat))
# print(df.isnull().sum())
# print(df.head())
df['Age'] = df['Age'].fillna(df['Age'].mean())
df['Fare'] = df['Fare'].fillna(df['Fare'].mean())
df = df.drop(columns=['Cabin'])
df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])

df = df.drop(columns=['Name'])
df = df.drop(columns=['Ticket'])
df = pd.get_dummies(df)
# print(df.isnull().sum())
# print(df.head())

# 스케일링
# print(train.head())
# print(train['Ticket'].unique())

# import sklearn.preprocessing
# print(dir(sklearn.preprocessing))

from sklearn.preprocessing import minmax_scale
df['Age'] = minmax_scale(df['Age'])
df['Fare'] = minmax_scale(df['Fare'])
# print(train.head())

train = df[:train.shape[0]]
test = df[train.shape[0]:]
# print(train.head())
# print(train.shape)
# print(test.head())
# print(test.shape)

# 모델 생성
# import sklearn.ensemble
# print(dir(sklearn.ensemble))

from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=2022)
# print(help(RandomForestClassifier))

# train_test_split
# import sklearn.model_selection
# print(dir(sklearn.model_selection))

from sklearn.model_selection import train_test_split
# print(help(train_test_split))
X_train, X_val, y_train, y_val = train_test_split(train, y_train, test_size=0.2)
# print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

# 모델 fit
model.fit(X_train, y_train)

# roc_auc
# import sklearn.metrics
# print(dir(sklearn.metrics))

from sklearn.metrics import roc_auc_score
# print(help(roc_auc_score))
print(roc_auc_score(model.predict(X_val), y_val))
print(model.score(X_val, y_val))

# predict
pred = model.predict(test)
# print(pred)

# 제출
# print(test.head(3))

submit = pd.DataFrame()
submit['PassengerId'] = test['PassengerId']
submit['Survived'] = pred

submit.to_csv('titanic_pred.csv', index=False)
# print(help(submit.to_csv))

submit = pd.read_csv('titanic_pred.csv')
print(submit.head())

y_test = pd.read_csv('./data/gender_submission.csv')
print(sum(submit['Survived'] == y_test['Survived']) / len(submit))

print(test.head(3))

0.8344280240831966
0.8324022346368715
   PassengerId  Survived
0          892         0
1          893         0
2          894         0
3          895         0
4          896         0
0.7894736842105263
   PassengerId  Pclass       Age  SibSp  Parch      Fare  Sex_female  \
0          892       3  0.430039      0      0  0.015282           0   
1          893       3  0.586622      1      0  0.013663           1   
2          894       2  0.774521      0      0  0.018909           0   

   Sex_male  Embarked_C  Embarked_Q  Embarked_S  
0         1           0           1           0  
1         0           0           0           1  
2         1           0           1           0  


C:\Users\hjk70\AppData\Local\Temp\ipykernel_21016\3536549714.py:81: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)


In [51]:
# 해답

# 시험환경 세팅 (코드 변경 X)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name
    
    if null_name != "":
        df[df == null_name] = np.nan
    
    X_train, X_test = train_test_split(df, test_size=0.2, random_state=2021)
    
    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[target])

    
    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[target])
    return X_train, X_test, y_train, y_test 
    
df = pd.read_csv("./data/train.csv")
X_train, X_test, y_train, y_test = exam_data_load(df, target='Survived', id_name='PassengerId')

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((712, 11), (179, 11), (712, 2), (179, 2))

In [52]:
y_train

,PassengerId,Survived
90,91,0
103,104,0
577,578,1
215,216,1
191,192,0
...,...,...
621,622,1
128,129,1
57,58,0
341,342,1


In [57]:
X_train.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
90,91,3,"Christmann, Mr. Emil",male,29.0,0,0,343276,8.0500,NaN,S
103,104,3,"Johansson, Mr. Gustaf Joel",male,33.0,0,0,7540,8.6542,NaN,S
577,578,1,"Silvey, Mrs. William Baird (Alice Munger)",female,39.0,1,0,13507,55.9000,E44,S
215,216,1,"Newell, Miss. Madeleine",female,31.0,1,0,35273,113.2750,D36,C
191,192,2,"Carbines, Mr. William",male,19.0,0,0,28424,13.0000,NaN,S


In [61]:
import pandas as pd

y = y_train["Survived"]

# sex만 원핫인코딩 됨
features = ["Pclass", "Sex", "SibSp", "Parch"]
X = pd.get_dummies(X_train[features])
test = pd.get_dummies(X_test[features])

In [59]:
X.head()

,Pclass,SibSp,Parch,Sex_female,Sex_male
90,3,0,0,0,1
103,3,0,0,0,1
577,1,1,0,1,0
215,1,1,0,1,0
191,2,0,0,0,1


In [62]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=200, max_depth=7, random_state=2021)
model.fit(X, y)
predictions = model.predict(test)

In [63]:
model.score(X, y)

0.8356741573033708

In [ ]:
output = pd.DataFrame({'PassengerId': X_test.PassengerId, 'Survived': predictions})
# output.to_csv('1234567.csv', index=False)